这是来自coursera的dataleakage技巧

https://www.coursera.org/learn/competitive-data-science/notebook/8MMqp/data-leakages

问题是：给定两个图片的id，判断这两个图片是否属于同一个类别

仅使用测试集就可以得到接近100%正确率的结果

到这里可以提交结果（以正确率判断）

https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages

In [1]:
# coding: utf8
import numpy as np
import pandas as pd
import scipy.sparse

In [2]:
test = pd.read_csv('predict_image_same_class_test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


In [3]:
concat_ids = pd.concat([test['FirstId'], test['SecondId']], axis=0)
print('count:', test['pairId'].count(), ', nunique:', concat_ids.nunique(), ', min:', concat_ids.min(), ', max:', concat_ids.max())

count: 368550 , nunique: 26325 , min: 0 , max: 26324


可以看到总共有36万多的测试数据，图片id去重后总共有26325个，0~26324

In [4]:
N = 26325

首先，两图片属于同一类的配对数量占所有图片一一配对得到的数量的很小一部分。如果我们有1000个类，每个类有N个图片，那么所有的配对可能有$\frac{1000N(1000N-1)}{2}$，而两个图片属于同一类的配对可能有$\frac{1000N(N-1)}{2}$。所以，这个可能性是很小的（当然取决于类别数量和N，不管怎么样属于同一类不会很多）

我们先提交一个submission，所有的预测结果是1，看看label=0/1的分布

In [5]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = 1
# submission.to_csv('submission.csv', index=False)
# get score 0.5

将这个csv提交后，得到了0.5的正确率，说明了两图片属于同一类的样本占到了一半，说明这个测试集不是完全随机的，一定存在某种生成规则

我们先计算图片之间的关联矩阵，如果他们的配对出现在了测试集中就是1，否则就是0

In [6]:
df = pd.DataFrame({
    'row': pd.concat([test['FirstId'], test['SecondId']]),
    'col': pd.concat([test['SecondId'], test['FirstId']]),
    'data': np.ones(test['pairId'].count() * 2),
})
df.drop_duplicates(inplace=True)
inc_mat = scipy.sparse.coo_matrix((df['data'], (df['row'], df['col'])), shape=(N, N))
inc_mat = inc_mat.tocsr()

我们利用关联矩阵，找到两个图片各自在关联矩阵中的行（也就是和其他图片是否一起出现过），进行逐个元素相乘并求和，有点相似度的感觉

In [7]:
rows_FirstId   = inc_mat[test['FirstId'].values, :]
rows_SecondId  = inc_mat[test['SecondId'].values, :]

f = rows_FirstId.multiply(rows_SecondId)
f = f.sum(axis=1)
f = np.array(f).flatten()

我们看一下得到的结果，可以看到基本都是14和20

In [8]:
pd.Series(f).value_counts().sort_index()

14.0    183279
15.0       852
19.0       546
20.0    183799
21.0         6
28.0        54
35.0        14
dtype: int64

这个结果表明测试集肯定是用了非常规则的生成方法，我们试试将>15的设置为1，其他为0

In [9]:
pred = f > 15

In [10]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

# submission.to_csv('submission.csv', index=False)

提交结果后，会得到接近1的正确率

上面是coursera中给出的一些解释，但是我个人认为，上面的关联矩阵只是一种巧合。但是不可否认，测试集一定是用了规则的生成方法，才会出现这么规律的结果，导致的数据泄露。然而，coursera并没有给出一个很好的理由说明为什么会这样，说实话我也没想到是什么原因。下面我随便进行了数据的展示，也发现了很多规律（虽然我也解释不了。。）。

In [11]:
test.groupby('FirstId').count()

,pairId,SecondId
FirstId,,
0,21,21
1,21,21
2,21,21
3,21,21
4,21,21
5,21,21
6,21,21
7,21,21
8,21,21


In [12]:
test.groupby('FirstId').count().groupby('SecondId').count()

,pairId
SecondId,
7,1755
8,1755
9,1755
10,1755
11,1755
12,1755
13,1755
14,1755
15,1755


In [13]:
pd.concat([test, test.rename(columns={'FirstId': 'SecondId', 'SecondId': 'FirstId'})]) \
    .groupby('FirstId').count().groupby('SecondId').count()

,pairId
SecondId,
21,14040
36,12285
